In [ ]:
import os
import shutil
from tqdm.notebook import tqdm
from PIL import Image

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import torchvision.transforms as transforms

from sklearn.model_selection import train_test_split

from IPython.display import clear_output
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
%matplotlib inline
from torchvision import datasets, models
from albumentations import (
    HorizontalFlip, ShiftScaleRotate, Blur, IAAAdditiveGaussianNoise, GaussNoise, 
    MotionBlur, MedianBlur, OneOf, Compose, Resize, HueSaturationValue, Normalize
)
from albumentations.pytorch.transforms import ToTensor
from torch import nn
import torchvision

In [ ]:
model = torch.load('/content/drive/MyDrive/ППШ проект/lstmmodelgpu_2.pth')

RuntimeError: ignored

In [ ]:
def proverka():
  class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
      def __getitem__(self, index):
          original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
          path = self.imgs[index][0]
          tuple_with_path = (original_tuple + (path,))
          return tuple_with_path
  ##########################################
  val_transforms = transforms.Compose([

      transforms.Resize((224, 224)),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ])

  ###########################################
  data_dir = '/content/drive/MyDrive/Proverka/photos/loaded/'
  test_dataset = ImageFolderWithPaths(os.path.join(data_dir, ''), val_transforms)

  batch_size = 1

  test_dataloader = torch.utils.data.DataLoader(
      test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
  ##########################################

  device_num = 0
  torch.cuda.set_device(device_num)
  device = f"cuda:{device_num}" if torch.cuda.is_available() else "cpu"
  print("Training on {}".format(device))
  ##########################################
  model.eval()

  test_predictions = []
  test_img_paths = []
  for inputs, labels, paths in tqdm(test_dataloader):
      inputs = inputs.to(device)
      labels = labels.to(device)
      with torch.set_grad_enabled(False):
          preds = model(inputs)
      test_predictions.append(
          torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
      test_img_paths.extend(paths)
      
  test_predictions = np.concatenate(test_predictions)
  return test_predictions
  ##################################################

In [ ]:
pip install python-telegram-bot

In [ ]:
#pip install pyTelegramBotAPI

In [ ]:
pip install pytelegrambotapi --upgrade

In [ ]:
pip install pytelegrambotapi

In [ ]:
import telebot

bot=telebot.TeleBot("1471020149:AAGnU5rEm9Wb4_WXzj_MI-_hGDsj80Zej-4")

@bot.message_handler(content_types=['photo'])
def handle_docs_photo(message):

    try:


        file_info = bot.get_file(message.photo[len(message.photo)-1].file_id)
        downloaded_file = bot.download_file(file_info.file_path)

        src='drive/MyDrive/Proverka/photos/loaded/'+file_info.file_path
        with open(src, 'wb') as new_file:
           new_file.write(downloaded_file)
        pred = proverka()
        if pred > 0.35:
             answer = 'Безопасно'
             shutil.move(src, 'drive/MyDrive/Proverka/safe')
        else:
             answer = 'Обнаружена опасность!'
             shutil.move(src, 'drive/MyDrive/Proverka/danger')
        bot.send_message(message.chat.id, answer)
    except Exception as e:
        shutil.rmtree('drive/MyDrive/Proverka/photos/loaded/photos/')
        os.mkdir('drive/MyDrive/Proverka/photos/loaded/photos')
        bot.reply_to(message, 'Простите, но во время обработки фотографии возникла ошибка. Пожалуйста, отправляйте только РАЗНЫЕ фотографии по одной в сообщении.' +
                     ' Если это сообщение выводится по ошибке, то скорее всего возникли проблемы на серверве. Просим вас ожидать, пока команда разработчиков их исправит.')

bot.polling()

ModuleNotFoundError: ignored